In [ ]:
# Now let's test the connection
from ai21 import AI21Client
import os

# paste your API key here or set it as an environment variable
API_KEY = os.environ.get("AI21_API_KEY")

# Make sure the API key is set
print(API_KEY)
client = AI21Client(api_key=API_KEY)

We will now create a Maestro run 